In [ ]:
Packages <- c("plm", "arrow", "panelvar", "dplyr")
invisible(lapply(Packages, library, character.only = TRUE))

In [ ]:
loincs_cofounder_potassium <- c('2157-6', #creatine kinase
                              '1920-8', #AST      
                              '2532-0', #LDH
                              '4542-7', #Haptoglobin 
                              '3084-1') #uric acid
bp_cofounder_potassium <- c('DIASTOLIC','SYSTOLIC')
loincs_potassium = c('2823-3', '2160-0')

In [ ]:
loinc1 <- '1920-8'
loinc2 <- '2160-0'
datafolder <- '/home/hoyinchan/blue/Data/data2021raw/'

In [ ]:
outputlab3 <- list()
#for (loinc1 in loincs_cofounder_potassium){
#for (loinc1 in bp_cofounder_potassium){
for (loinc1 in c(loincs_cofounder_potassium, bp_cofounder_potassium)){
    for (loinc2 in loincs_potassium){
        key <- paste0(loinc1,loinc2)
        path1 <- paste0(datafolder,'outputlab',loinc1,loinc2,'.parquet')
        outputlab3[[key]] <- arrow::read_parquet(path1)
    }
}

In [ ]:
#for (loinc1 in loincs_cofounder_potassium){
#for (loinc1 in bp_cofounder_potassium){    
for (loinc1 in c(loincs_cofounder_potassium, bp_cofounder_potassium)){
    for (loinc2 in loincs_potassium){
        key <- paste0(loinc1,loinc2)
        print(paste(loinc1, loinc2))        
        pGrunfeld_unbal <- pdata.frame(outputlab3[[key]], index = c("ID", "DAYS_SINCE_ADMIT"))        
        tryCatch({
            print(purtest(pGrunfeld_unbal$res, pmax=1, test ="madwu"))
        }, error=function(e){})
        tryCatch({
            print(cipstest(pGrunfeld_unbal$res, pmax=1))
        }, error=function(e){})
        
    }
}

In [ ]:
# pGrunfeld_unbal <- pdata.frame(outputlab3, index = c("ID", "DAYS_SINCE_ADMIT"))
# print(purtest(pGrunfeld_unbal$res, pmax=3, test ="madwu"))

In [ ]:
loinc1 <- 'SYSTOLIC'
loinc2 <- '2160-0'

In [ ]:
key <- paste0(loinc1,loinc2)
testdf <- outputlab3[[key]]
names(testdf)[names(testdf) == 'ID'] <- 'id'
names(testdf)[names(testdf) == 'DAYS_SINCE_ADMIT'] <- 'year'
names(testdf)[names(testdf) == loinc1] <- 'expenditures'
names(testdf)[names(testdf) == loinc2] <- 'revenues'

#testdf$id <- testdf$id+1
#testdf$year <- testdf$year+1980
testdf$id <- as.factor(testdf$id)
testdf$year <- as.factor(testdf$year)
testdf <- testdf %>% select(c('id', 'year', 'expenditures', 'revenues'))

#testdf <- testdf %>% filter(id %in% c(1,2,3,4))

In [ ]:
# Sample 10 unique IDs
sampled_ids <- sample(unique(testdf$id), 20)
# Get all rows that match these IDs
testdf <- subset(testdf, id %in% sampled_ids)

In [ ]:
# data("Dahlberg")
# Dahlberg2 <- Dahlberg %>% filter(id %in% c(114,115,123))
# Dahlberg3 <- Dahlberg2[1:23,]
# result <- pvargmm(dependent = c("expenditures", "revenues"), data = Dahlberg3, lag=1, steps=c("twostep"))
# Andrews_Lu_MMSC(result)
# stabr <-stability(result)
# print(stabr)

In [ ]:
data("Dahlberg")

In [ ]:
Dahlberg3 <- Dahlberg[1:1,]
Dahlberg3 <- Dahlberg3 %>% select(-grants)

In [ ]:
Dahlberg3[dim(testdf)[1],]<-NA

In [ ]:
Dahlberg3$id <- testdf$id
Dahlberg3$year <- testdf$year
Dahlberg3$expenditures <- testdf$expenditures
Dahlberg3$revenues <- testdf$revenues

In [ ]:
resulttime <- system.time({
    result <- pvargmm(dependent = c("expenditures", "revenues"), data = Dahlberg3, lag=3, steps=c("twostep"), panel_identifier = c("id","year"), collapse=TRUE, max_instr_dependent_vars=10)
})
print(resulttime)

In [ ]:
result